In [5]:
import csv
import cv2
import glob
import json
import numpy as np
import pandas as pd
import time

from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.core import Reshape
from keras.models import Sequential
from keras.optimizers import SGD
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tqdm import tqdm

print ('Loading training data...')
time_load_start = time.time()         # Returns the number of ticks after a certain event.



Loading training data...


In [14]:
# Load jpg images to get image_array.
# training_images = glob.glob('./training_data/*_cnn/*.jpg')
# image_array = np.array([cv2.imread(name, cv2.IMREAD_GRAYSCALE) for name in tqdm(training_images)], dtype=np.float64)


# Load training data .npz to get label_array, unpacking what's in the saved .npz files.
# label_array = np.zeros((1, 3), 'float')
training_data = glob.glob('training_data/*.npz')        # Finds filename matching specified path or pattern.

image_array = None
data_array = None
for single_npz in training_data:                        # single_npz == one array representing one array of saved image data and user input label for that image.
    with np.load(single_npz) as data:
        train_image_temp = data['train']                      # returns the training data image array assigned to 'train' argument created during np.savez step in 'collect_training_data.py'
        print('train data shape: ', train_data_temp.shape)
        train_labels_temp = data['train_labels']        # returns the training user input data array assigned to 'train_labels' argument created during np.savez step in 'collect_training_data.py'
        print ('Original labels shape:', train_labels_temp.shape)
    label_array = np.array([label for label in tqdm(train_labels_temp)], dtype=np.float64)
    image_array = np.array([data for data in tqdm(train_image_temp)], dtype=np.float64)
    image_array = np.reshape(image_array, (len(image_array), 120, 320, 1))
    print(label_array.shape)
    print(image_array.shape)



train data shape:  (571, 38400)
Original labels shape: (540, 4)


100%|████████████████████████████████████████████████████████████████████████████| 540/540 [00:00<00:00, 179955.84it/s]


(540, 4)
(540, 120, 320, 1)
train data shape:  (571, 38400)
Original labels shape: (648, 4)


100%|████████████████████████████████████████████████████████████████████████████| 648/648 [00:00<00:00, 259095.23it/s]


(648, 4)
(648, 120, 320, 1)
train data shape:  (571, 38400)
Original labels shape: (571, 4)


100%|████████████████████████████████████████████████████████████████████████████| 571/571 [00:00<00:00, 285282.62it/s]


(571, 4)
(571, 120, 320, 1)


In [15]:
X = image_array
y = label_array
print ('Shape of feature array: ', X.shape)
print (type(X))
print ('Shape of label array: ', y.shape)
print (type(y))

Shape of feature array:  (571, 120, 320, 1)
<class 'numpy.ndarray'>
Shape of label array:  (571, 4)
<class 'numpy.ndarray'>
(456, 120, 320, 1)
(115, 120, 320, 1)
(456, 4)
(115, 4)


In [20]:
# # Normalize with l2 (not gonna use this...)
# X = preprocessing.normalize(X, norm='l2')

# Normalize from 0 to 1
X = X / 255.
# for each in tqdm(X):
#     each/255.

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)

(456, 120, 320, 1)
(115, 120, 320, 1)
(456, 4)
(115, 4)


In [23]:
model = Sequential()

time_load_end = time.time()
time_load_total = time_load_end - time_load_start
print ('Total time taken to load image data:', time_load_total, 'seconds')


# Get start time of Training
time_training_start = time.time()

print( 'Training...')

# CONVOLUTION

# When using THEANO backend
# model.add(Reshape((1, 120, 320), input_shape=(120, 320)))

# # When using TENSORFLOW backend
# model.add(Reshape((120, 320, 1), input_shape=(120, 320)))

model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(120 ,320, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(30, init='uniform'))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(4, init='uniform'))
model.add(Activation('softmax'))


model.summary()



Total time taken to load image data: 1027.2736253738403 seconds
Training...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_5 (Convolution2D)  (None, 120, 320, 32)  320         convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
activation_7 (Activation)        (None, 120, 320, 32)  0           convolution2d_5[0][0]            
____________________________________________________________________________________________________
maxpooling2d_5 (MaxPooling2D)    (None, 60, 160, 32)   0           activation_7[0][0]               
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)  (None, 58, 158, 32)   9248        maxpooling2d_5[0][0]             
_______________

In [24]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])


# Fit the model
model.fit(X_train, y_train,
          nb_epoch=20,
          batch_size=1000,
          validation_data=(X_test, y_test))


Train on 456 samples, validate on 115 samples
Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor with shape[456,32,120,320]
	 [[Node: Conv2D_4 = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_convolution2d_input_2_0/_9, convolution2d_5_W/read)]]
	 [[Node: mul_5/_37 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_15_mul_5", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Conv2D_4', defined at:
  File "d:\program files\python35\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\program files\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\program files\python35\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "d:\program files\python

In [ ]:


# Get end time of Training
time_training_end = time.time()
time_training_total = time_training_end - time_training_start
print( '')
print( 'Total time taken to train model:', time_training_total, 'seconds')

# Evalute trained model on TEST set
print( '')
print ('Evaluation of model on test holdout set:')
score = model.evaluate(X_test, y_test, batch_size=1000)
loss = score[0]
accuracy = score[1]
print ''
print 'Loss score: ', loss
print 'Accuracy score: ', accuracy

# Save model as h5
timestr = time.strftime('%Y%m%d_%H%M%S')
filename_timestr = 'nn_{}.h5'.format(timestr)
model.save('nn_h5/nn_{}.h5'.format(timestr))

# Save parameters to json file
json_string = model.to_json()
with open('./logs/nn_params_json/nn_{}.json'.format(timestr), 'w') as new_json:
    json.dump(json_string, new_json)

# Save training results to csv log
row_params = [str(training_data)[-33:-2], filename_timestr, loss, accuracy]
with open('./logs/log_nn_training.csv','a') as log:
    log_writer = csv.writer(log)
    log_writer.writerow(row_params)
